# Assignment 3 on Natural Language Processing

## Date : 30th Sept, 2020

### Instructor : Prof. Sudeshna Sarkar

### Teaching Assistants : Alapan Kuila, Aniruddha Roy, Anusha Potnuru, Uppada Vishnu

The central idea of this assignment is to use Naive Bayes classifier and LSTM based classifier and compare the models by accuracy on IMDB dataset.



Please submit with outputs. 

In [1]:
import re
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import keras
from sklearn.metrics import classification_report

In [2]:
#Load the IMDB dataset. You can load it using pandas as dataframe
df = pd.read_csv("IMDB Dataset.csv")

In [3]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [4]:
df.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


In [5]:
df['sentiment'].value_counts()

positive    25000
negative    25000
Name: sentiment, dtype: int64

# Preprocessing
PrePrecessing that needs to be done on lower cased corpus

1. Remove html tags
2. Remove URLS
3. Remove non alphanumeric character
4. Remove Stopwords
5. Perform stemming and lemmatization

You can use regex from re. 

In [6]:
def lower_case(raw_text):
  return raw_text.lower()

In [7]:
from bs4 import BeautifulSoup
def clean_html(raw_text):
  clean_text = BeautifulSoup(raw_text, "lxml").text
  return clean_text

In [8]:
def clean_URLs(raw_text):
  clean_text = re.sub(r'^https?:\/\/.*[\r\n]*', '', raw_text, flags=re.MULTILINE)
  return clean_text

In [9]:
def clean_non_alpha(raw_text):
  return re.sub(r'[^a-zA-z0-9\s]', '', raw_text)

In [10]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
from nltk.corpus import stopwords
from nltk.tokenize.toktok import ToktokTokenizer
tokenizer=ToktokTokenizer()

stopwords_list=stopwords.words('english')
#print(stopwords_list)

def remove_stopwords(raw_text):
  tokens = tokenizer.tokenize(raw_text)
  clean_tokens = [token for token in tokens if token not in stopwords_list]
  clean_text = ' '.join(clean_tokens) 
  return clean_text

In [12]:
def stemming(text):
  ps=nltk.porter.PorterStemmer()
  text= ' '.join([ps.stem(word) for word in text.split()])
  return text

In [13]:
df['review']=df['review'].apply(lower_case)
df['review']=df['review'].apply(clean_html)
df['review']=df['review'].apply(clean_URLs)
df['review']=df['review'].apply(clean_non_alpha)
df['review']=df['review'].apply(remove_stopwords)
df['review']=df['review'].apply(stemming)

In [14]:
#df['review'][0]

In [15]:
# Print Statistics of Data like avg length of sentence , proposition of data w.r.t class labels
length = [0] * df.shape[0]
i=0
for sentence in df['review']:
  tokens = tokenizer.tokenize(sentence)
  length[i] = len(tokens)
  i+=1

avg_length = sum(length)/len(length)
max_length = max(length)
min_length = min(length)

print("avg length - ", avg_length)
print("max length - ", max_length)
print("min length - ", min_length)

avg length -  119.8392
max length -  1429
min length -  0


# Naive Bayes classifier

In [16]:
# get reviews column from df
reviews = df['review']

# get labels column from df
labels = df['sentiment']

In [17]:
# Use label encoder to encode labels. Convert to 0/1
encoder = LabelEncoder()
encoded_labels = encoder.fit_transform(labels)

print(encoded_labels[:10])

[1 1 1 0 1 1 1 0 0 1]


In [18]:
# Split the data into train and test (80% - 20%). 
# Use stratify in train_test_split so that both train and test have similar ratio of positive and negative samples.

train_reviews, test_reviews, train_labels, test_labels = train_test_split(reviews, encoded_labels, test_size=0.2, stratify=encoded_labels)

# train_sentences, test_sentences, train_labels, test_labels

train_reviews = list(train_reviews)
test_reviews = list(test_reviews)
train_labels = list(train_labels)
test_labels = list(test_labels)

Here there are two approaches possible for building vocabulary for the naive Bayes.
1. Take the whole data (train + test) to build the vocab. In this way while testing there is no word which will be out of vocabulary.
2. Take the train data to build vocab. In this case, some words from the test set may not be in vocab and hence one needs to perform smoothing so that one the probability term is not zero.
 
You are supposed to go by the 2nd approach.
 
Also building vocab by taking all words in the train set is memory intensive, hence you are required to build vocab by choosing the top 2000 - 3000 frequent words in the training corpus.

> $ P(x_i | w_j) = \frac{ N_{x_i,w_j}\, +\, \alpha }{ N_{w_j}\, +\, \alpha*d} $


$N_{x_i,w_j}$ : Number of times feature $x_i$ appears in samples of class $w_j$

$N_{w_j}$ : Total count of features in class $w_j$

$\alpha$ : Parameter for additive smoothing. Here consider $\alpha$ = 1

$d$ : Dimentionality of the feature vector  $x = [x_1,x_2,...,x_d]$. In our case its the vocab size.






In [19]:
from sklearn.feature_extraction.text import CountVectorizer
# Use Count vectorizer to get frequency of the words
'''
max_features parameter : If not None, build a vocabulary that only consider the top max_features ordered by term frequency across the corpus.
vec = CountVectorizer(max_features = 3000)
X = vec.fit_transform(Sentence_list)
'''

vec = CountVectorizer(max_features = 3000)
train_freq = vec.fit_transform(train_reviews).toarray()
dictionary = vec.get_feature_names()

print(train_freq.shape)
#print(dictionary)

(40000, 3000)


In [20]:
N_pos = train_labels.count(1)
N_neg = train_labels.count(0)
print(N_pos, N_neg)

20000 20000


In [21]:
pos_dict = {}
neg_dict = {}

for j in range(len(dictionary)):
  pos_count=0
  neg_count=0
  for i in range(train_freq.shape[0]):
    if(train_labels[i]==1):
      pos_count += train_freq[i][j]
    else:
      neg_count += train_freq[i][j]
  pos_dict[dictionary[j]] = pos_count
  neg_dict[dictionary[j]] = neg_count


In [22]:
# Use laplace smoothing for words in test set not present in vocab of train set
# Build the model. Don't use the model from sklearn

log_pos = [0] * len(test_reviews)
log_neg = [0] * len(test_reviews)

for i in range(len(test_reviews)):
  tokens = tokenizer.tokenize(test_reviews[i])
  for token in tokens:
    if token in dictionary:
      prob_pos = (pos_dict[token]+1)/(N_pos+len(dictionary))
      prob_neg = (neg_dict[token]+1)/(N_neg+len(dictionary))
    else:
      prob_pos = 1 / (N_pos+len(dictionary))
      prob_neg = 1 / (N_neg+len(dictionary))
    log_pos[i] += np.log(prob_pos)
    log_neg[i] += np.log(prob_neg)


In [23]:
print(len(log_pos))
print(len(log_neg))

10000
10000


In [24]:
# Test the model on test set and report Accuracy
pred_labels = [0] * len(test_labels)

for i in range(len(test_labels)):
  pred_labels[i]=int(log_pos[i]>=log_neg[i])

#print(pred_labels)

accuracy = sum(x == y for x, y in zip(pred_labels, test_labels)) / len(test_labels)
print(accuracy) 


0.8388


# *LSTM* based Classifier

Use the above train and test splits.

In [28]:
# Hyperparameters of the model
vocab_size = len(dictionary) # choose based on statistics
oov_tok = '<OOK>'
embedding_dim = 100
max_length = 1500 # choose based on statistics, for example 150 to 200
padding_type='post'
trunc_type='post'

In [29]:
# tokenize sentences
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_reviews)
word_index = tokenizer.word_index

# convert train dataset to sequence and pad sequences
train_sequences = tokenizer.texts_to_sequences(train_reviews)
train_padded = pad_sequences(train_sequences, padding='post', maxlen=max_length)

# convert Test dataset to sequence and pad sequences
test_sequences = tokenizer.texts_to_sequences(test_reviews)
test_padded = pad_sequences(test_sequences, padding='post', maxlen=max_length)

In [45]:
# model initialization
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

# compile model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# model summary
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 1500, 100)         300000    
_________________________________________________________________
bidirectional_4 (Bidirection (None, 128)               84480     
_________________________________________________________________
dense_9 (Dense)              (None, 24)                3096      
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 25        
Total params: 387,601
Trainable params: 387,601
Non-trainable params: 0
_________________________________________________________________


In [46]:
num_epochs = 5
history = model.fit(train_padded, np.array(train_labels), 
                    epochs=num_epochs, verbose=1,
                    validation_split=0.1) 

Epoch 1/10
1125/1125 [==============================] - 129s 114ms/step - loss: 0.4132 - accuracy: 0.8176 - val_loss: 0.3517 - val_accuracy: 0.8512
Epoch 2/10
1125/1125 [==============================] - 128s 114ms/step - loss: 0.3401 - accuracy: 0.8624 - val_loss: 0.3444 - val_accuracy: 0.8555
Epoch 3/10
1125/1125 [==============================] - 127s 113ms/step - loss: 0.2869 - accuracy: 0.8875 - val_loss: 0.3200 - val_accuracy: 0.8665
Epoch 4/10
1125/1125 [==============================] - 128s 114ms/step - loss: 0.2577 - accuracy: 0.8991 - val_loss: 0.3243 - val_accuracy: 0.8702
Epoch 5/10
1125/1125 [==============================] - 128s 113ms/step - loss: 0.2334 - accuracy: 0.9102 - val_loss: 0.3348 - val_accuracy: 0.8635
Epoch 6/10
1125/1125 [==============================] - 128s 113ms/step - loss: 0.2121 - accuracy: 0.9193 - val_loss: 0.3406 - val_accuracy: 0.8687
Epoch 7/10
1125/1125 [==============================] - 128s 113ms/step - loss: 0.1892 - accuracy: 0.9304 - val_

In [47]:
# Calculate accuracy on Test data
'''
prediction = model.predict(test_padded)

'''
# Get probabilities
probabilities = model.predict(test_padded).squeeze()

# Get labels based on probability 1 if p>= 0.5 else 0
pred_labels = probabilities>=0.5

# Accuracy : one can use classification_report from sklearn
accuracy = sum(x == y for x, y in zip(pred_labels, test_labels)) / len(test_labels)
print(accuracy*100)

86.31


## Get predictions for random examples

In [41]:
# reviews on which we need to predict
sentence = ["The movie was very touching and heart whelming", 
            "I have never seen a terrible movie like this", 
            "the movie plot is terrible but it had good acting"]

# convert to a sequence
sequences = tokenizer.texts_to_sequences(sentence)

# pad the sequence
padded = pad_sequences(sequences, padding='post', maxlen=1500)

# Get probabilities
prob = model.predict(padded).squeeze()
print(prob)

# Get labels based on probability 1 if p>= 0.5 else 0
pred_labels = (prob>=0.5).squeeze()
print(pred_labels)

[0.6004889 0.6055787 0.5217633]
[ True  True  True]
